In [41]:
import pickle
import torch

from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

import networkx as nx
import numpy as np
import pandas as pd
import deepchem as dc

from tqdm.notebook import tqdm

In [31]:
data = pd.read_csv("B3DB/B3DB/B3DB_classification.tsv", sep="\t").reset_index()
display(data)

,index,NO.,compound_name,IUPAC_name,SMILES,CID,logBB,BBB+/BBB-,Inchi,threshold,reference,group,comments
0,0,1,sulphasalazine,2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,5339.0,-2.69,BBB-,InChI=1S/C18H14N4O5S/c23-16-9-6-13(11-15(16)18...,NaN,R2|R2|R25|R46|,A,NaN
1,1,2,moxalactam,7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,3889.0,-2.52,BBB-,InChI=1S/C20H20N6O9S/c1-25-19(22-23-24-25)36-8...,NaN,R25|,A,NaN
2,2,3,clioquinol,5-chloro-7-iodoquinolin-8-ol,Oc1c(I)cc(Cl)c2cccnc12,2788.0,-2.40,BBB-,InChI=1S/C9H5ClINO/c10-6-4-7(11)9(13)8-5(6)2-1...,NaN,R18|R26|R27|,A,NaN
3,3,4,bbcpd11 (cimetidine analog) (y-g13),2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...,CCNC(=NCCSCc1ncccc1Br)NC#N,14022517.0,-2.15,BBB-,InChI=1S/C12H16BrN5S/c1-2-15-12(18-9-14)17-6-7...,NaN,R2|R2|R8|R40|R2|R2|R2|R2|R18|R21|R25|R25|R26|R...,A,NaN
4,4,5,schembl614298,"(2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...",CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,18595497.0,-2.15,BBB-,InChI=1S/C23H27NO9/c1-24-7-6-23-10-3-4-12(25)2...,NaN,R25|,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7802,7802,7803,antazoline,"n-benzyl-n-(4,5-dihydro-1h-imidazol-2-ylmethyl...",c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,2200.0,NaN,BBB-,InChI=1S/C17H19N3/c1-3-7-15(8-4-1)13-20(14-17-...,NaN,R14|R23|R50|,D,NaN
7803,7803,7804,emedastine,"1-(2-ethoxyethyl)-2-(4-methyl-1,4-diazepan-1-y...",CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,3219.0,NaN,BBB+,InChI=1S/C17H26N4O/c1-3-22-14-13-21-16-8-5-4-7...,NaN,R15|R23|R50|,D,NaN
7804,7804,7805,ketotifen,2-(1-methylpiperidin-4-ylidene)-6-thiatricyclo...,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,3827.0,NaN,BBB+,InChI=1S/C19H19NOS/c1-20-9-6-13(7-10-20)18-15-...,NaN,R15|R27|R50|,D,NaN
7805,7805,7806,milrinone,6-methyl-2-oxo-5-pyridin-4-yl-1h-pyridine-3-ca...,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,4197.0,NaN,BBB-,InChI=1S/C12H9N3O/c1-8-11(9-2-4-14-5-3-9)6-10(...,NaN,R15|R23|R27|,D,NaN


In [32]:
featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)


In [51]:
dataset = []
for index, row in tqdm(data.iterrows(), total= data.shape[0]):
    f = featurizer.featurize(row["SMILES"])
    if f.shape == (1, 0):
        continue
    d = f[0].to_pyg_graph()
    d.y = 1 if row['BBB+/BBB-'] == 'BBB+' else 0
    dataset.append(d)
    
    
    
print(len(dataset))

  0%|          | 0/7807 [00:00<?, ?it/s]

Failed to featurize datapoint 0, [Li]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [Kr]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, O. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, C. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [Ar]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [Xe]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [

7800


In [53]:
with open("dataset.dat", "wb") as f:
    pickle.dump(dataset, f)

In [3]:
# *****how are graph labels being sent across? 
num_features = 2
def nx_to_pyg(gr):
    
    x = np.zeros((len(gr.nodes()),num_features))
    i = 0
    for v in gr.nodes().data():
        x[i,0] = v[1]['atomic_num']
        x[i,1] = v[1]['is_aromatic']
        i += 1
    x = torch.Tensor(x)

    edge_index = []
    for e in gr.edges().data():
        edge_index.append([e[0], e[1]])
    edge_index = torch.Tensor(edge_index).t().contiguous()

    y = gr['y']
        
    return Data(x=x, edge_index=edge_index, y=y)

In [91]:
nx_to_pyg(gr_list)

[Data(x=[5, 2], edge_index=[2, 8])]

In [5]:
def from_networkx(G):
    G = nx.convert_node_labels_to_integers(G)
    edges = list(G.edges)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    data = defaultdict(list)
    data['edge_index'] = edge_index.view(2, -1)
    for key, value in G.graph.items():
        data[key] = value
    xs = []
    for i, (_, node_data) in enumerate(G.nodes(data=True)):
        for key, value in node_data.items():
            data[key].append(value)
    for i, (_, _, edge_data) in enumerate(G.edges(data=True)):
        for key, value in edge_data.items():
            data[key].append(value)
                            
    for key in ["atomic_num", "is_aromatic"]:
        xs.append(data[key])
        data['x'] = torch.tensor(xs).t().contiguous()

    return Data.from_dict(data)

In [49]:
def networkx_2_tensor(G):
    G = nx.convert_node_labels_to_integers(G)
    edges = list(G.edges)
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_index = edge_index.view(2, -1)
    
    for key, value in G.graph.items():
        data[key] = value
    
    for i, (_, node_data) in enumerate(G.nodes(data=True)):
        data[key] = []
        for key, value in node_data.items():
            data[key] = data[key].append(value)
    
    edge_attr = []
    for i, (_, _, edge_data) in enumerate(G.edges(data=True)):
        attr_i = []
        for key, value in edge_data.items():
            attr_i.append(value)
        edge_attr.append(attr_i)
    
    edge_attr = torch.tensor(edge_attr)
    
    xs = []
    for key in ["atomic_num", "is_aromatic"]:
        xs.append(data[key])
    x = torch.tensor(xs).t().contiguous()

    return Data.from_dict(data)

In [50]:
def mol_to_graph(mol, data=None):
    if data:
        G = nx.Graph(
            y=1 if data['BBB+/BBB-']=="BBB+" else 0, 
        )
    else:
        G = nx.Graph()
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum(), is_aromatic=atom.GetIsAromatic())

    for bond in mol.GetBonds():
        bond_weight = weight_map[str(bond.GetBondType())]
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), edge_attr=bond_weight)
    
    return G

In [51]:
weight_map = {
    'SINGLE': 1,
    'AROMATIC': 1.5,
    'DOUBLE': 2,
    'TRIPLE': 3
}

In [52]:
import pandas as pd
from rdkit import Chem
from collections import defaultdict

In [53]:
data = pd.read_csv("B3DB/B3DB/B3DB_classification.tsv", sep="\t")

In [54]:
len(data)

7807

In [55]:
row = data.sample().iloc[0]
mol = Chem.MolFromSmiles(row['SMILES'])
G = mol_to_graph(mol, dict(row))
g_tensor = networkx_2_tensor(G)
# g_tensor = nx_to_pyg(G)
g_tensor = Data(x=g_tensor.x, edge_index=g_tensor.edge_index, y= torch.tensor(np.ones(g_tensor.x.shape[0]) * g_tensor.y))

KeyError: 'atomic_num'

In [56]:
g_tensor

Data(x=[2], edge_index=[2, 32], y=[2])

In [ ]:
config = {
    'edge_features': False,
    'node_labels': True, # true if the y tensor should be of shape
}

In [ ]:
def build_dataset(config):
    
    data_list_fix = []

    for index, row in data.iterrows():
        mol = Chem.MolFromSmiles(row['SMILES'])
        G = mol_to_graph(mol, dict(row))
        g_tensor = from_networkx(G)
        # g_tensor = nx_to_pyg(G)
        g_tensor = Data(x=g_tensor.x, edge_index=g_tensor.edge_index, y= torch.tensor(np.ones(g_tensor.x.shape[0]) * g_tensor.y))
        data_list_fix.append(g_tensor)

In [24]:
data_list = []

for index, row in data.iterrows():
    mol = Chem.MolFromSmiles(row['SMILES'])
    G = mol_to_graph(mol, dict(row))
    g_tensor = from_networkx(G)
    # g_tensor = nx_to_pyg(G)
    g_tensor = Data(x=g_tensor.x, edge_index=g_tensor.edge_index, y= torch.tensor(np.ones(g_tensor.x.shape[0]) * g_tensor.y))
    data_list.append(g_tensor)

In [25]:
data_list[0].y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], dtype=torch.float64)

In [26]:
torch.save(data_list, 'tensor_list.pt')

In [3]:
data_list = torch.load('tensor_list.pt')

In [4]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader